# 9장 엑셀과 파이썬을 이용한 통계 데이터 분석

## 9.1 통계 데이터 분석 기본

### 기본 통계량 이해

### 기본 통계량 구하기

**[9장: 518페이지]**

In [ ]:
import pandas as pd

#  CSV 데이터 파일 경로
folder = 'C:/myPyExcel/data/ch09/'
csv_file = folder + '지점별_일일_판매량.csv' 

# CSV 파일 읽기, CSV 파일의 '날짜' 열을 DataFrame 데이터의 index로 지정
df = pd.read_csv(csv_file, index_col='날짜')

# DataFrame 데이터 중 앞의 일부만 출력
df.head()

**[9장: 519페이지]**

In [ ]:
df_stat = df.describe()
df_stat

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# 한글 폰트를 위한 설정
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

# 박스 그래프 그리기
ax = df.plot.box(y=["A지점", "B지점", "C지점"], showmeans=True) # 박스 그래프
ax.set_ylabel("일일 판매량", fontsize=15)                        # y축 라벨을 추가
ax.set_title("지점별 일일 판매량 분포", fontsize=20)             # 그래프 제목을 추가

# 엑셀 파일에 추가할 그래프 이미지 파일
image_file = "C:/myPyExcel/figures/지점별_일일_판매량_분포.png" 

# 그래프를 이미지 파일로 저장
plt.savefig(image_file, dpi=300) 
plt.show()

**[9장: 520~521페이지]**

In [ ]:
import pandas as pd

# 저장할 엑셀 파일 경로
folder = 'C:/myPyExcel/data/ch09/'
excel_file= folder + '지점별_일일_판매량_기본통계량_박스그래프.xlsx' 
sheet = 'Sheet1'

# 쓰기 엔진을 xlsxwriter로 지정해 판다스의 ExcelWriter 객체 생성
excel_writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')

# DataFrame 데이터(df)를 생성한 엑셀 객체에 쓰기(시트 이름을 지정)
df.to_excel(excel_writer, sheet_name=sheet) # index를 포함

# 워크시트에 위치를 지정해 DataFrame 데이터(df_stat) 쓰기
row = 0 # 행 위치
col = len(df.columns) + 2 # 열 위치
df_stat.to_excel(excel_writer, startrow=row, startcol=col, sheet_name=sheet)

# 생성한 ExcelWriter 객체(excel_writer)를 이용해 워크시트 객체 생성
worksheet = excel_writer.sheets[sheet]

# 워크시트 객체(worksheet)에 이미지가 들어갈 위치를 지정해 이미지 넣기
row = len(df_stat.index) + 2  # 행 위치
col = len(df.columns) + 2     # 열 위치
worksheet.insert_image(row, col, image_file, {'x_scale': 0.6, 'y_scale': 0.6})

# 객체를 닫고 엑셀 파일로 저장
excel_writer.save()

print("생성 파일:", excel_file) # 생성한 파일 이름 출력

**[9장: 522페이지]**

In [ ]:
import pandas as pd

#  데이터 파일의 전체 경로
folder = 'C:/myPyExcel/data/ch09/'
csv_file = folder + '2019년_지점별_일일_판매량.csv' 

# CSV 파일 읽기, CSV 파일의 '날짜' 열을 DataFrame 데이터의 index로 지정
df = pd.read_csv(csv_file)

# DataFrame 데이터 출력
df

**[9장: 523페이지]**

In [ ]:
df.info()

**[9장: 524페이지]**

In [ ]:
df['날짜'] = pd.to_datetime(df['날짜']) 
df.info()

In [ ]:
df_months = [group for name, group in df.groupby(pd.Grouper(key='날짜',freq='M'))]
df_months[0].head() # 1월 데이터 일부를 출력

**[9장: 525페이지]**

In [ ]:
df_months[1].head() # 2월 데이터 일부를 출력

**[9장: 525~527페이지]**

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# 한글 폰트를 위한 설정
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

        
# 월별 데이터, 기본 통계량, 박스 그래프를 셀 파일에 쓰는 함수
# 입력 인수: 엑셀 파일 경로, DataFrame 데이터 리스트
# 반환: 없음
def write_stat_data_box_plot_to_excel_sheet(write_excel_file, df_months):
    # 쓰기 엔진을 xlsxwriter로 지정해 판다스의 ExcelWriter 객체 생성
    excel_writer = pd.ExcelWriter(write_excel_file, engine='xlsxwriter') 

    for month_num, df_month in enumerate(df_months): # 월별로 수행
        # 기본 통계량 구하기
        df_month_stat = df_month.describe()
        
        # 시트 이름 생성
        sheet = "{0:2d}월".format(month_num + 1)
        
        # '날짜'열을 datetime 형식에서 문자열로 변환
        df_month['날짜'] = df_month['날짜'].astype(str) 
        
        # DataFrame 데이터의 '날짜'열을 index로 지정
        df_month = df_month.set_index(['날짜']) 
        
        # DataFrame 데이터를 생성한 엑셀 객체에 쓰기(시트이름을 지정)       
        df_month.to_excel(excel_writer, sheet_name = sheet) # index를 포함

        # 워크시트에 위치를 지정해 기본 통계량 데이터(df_month_stat) 쓰기
        row = 0 # 행 위치
        col = len(df_month.columns) + 2  # 열 위치
        df_month_stat.to_excel(excel_writer, sheet_name=sheet,
                               startrow=row, startcol=col)
        
        # ------ 박스 그래프 그리고 이미지 파일로 저장하기 --------------
        ax = df_month.plot.box(y=["A지점", "B지점", "C지점"], 
                               showmeans=True) # 박스 그래프
        ax.set_ylabel("일일 판매량", fontsize=15)  # y 라벨 추가
        ax.set_title("지점별 일일 판매량 분포", fontsize=20) # 그래프 제목 추가

        # 박스 그래프의 이미지 파일 경로
        folder = "C:/myPyExcel/figures/"
        image_file = folder + "지점별_일일_판매량_분포_{0:02d}월.png".format(month_num + 1) 

        plt.savefig(image_file, dpi=300) # 그래프를 이미지 파일로 저장
        plt.close() # 화면에 그래프를 표시하지 않고 닫기
        # ----------------------------------------------------------------
        
        # 생성한 ExcelWriter 객체(excel_writer)를 이용해 워크시트 객체 생성
        worksheet = excel_writer.sheets[sheet]

        # 워크시트 객체(worksheet)에 이미지가 들어갈 위치를 지정해 이미지 넣기
        row = len(df_month_stat.index) + 2  # 행 위치
        col = len(df_month.columns) + 2  # 열 위치
        worksheet.insert_image(row, col, image_file, 
                               {'x_scale': 0.6, 'y_scale': 0.6})

    # 객체를 닫고 엑셀 파일로 저장
    excel_writer.save()

**[9장: 527페이지]**

In [ ]:
# 데이터 파일에서 DataFrame 데이터 가져오기
folder = 'C:/myPyExcel/data/ch09/'
csv_file = folder + '2019년_지점별_일일_판매량.csv' 
df = pd.read_csv(csv_file)

# DataFrame 데이터 중 '날짜'열의 데이터 타입을 datetime 형식으로 변환
df['날짜']= pd.to_datetime(df['날짜']) 

# 월별로 DataFrame 데이터 묶기(그룹핑)
df_months = [group for name, group in df.groupby(pd.Grouper(key='날짜',freq='M'))]

# 저장할 엑셀 파일 경로
folder = 'C:/myPyExcel/data/ch09/'
excel_file = folder + '2019년_지점별_일일_판매량_기본통계량_박스그래프.xlsx' 

# DataFrame 데이터, 기본 통계량, 박스 그래프를 엑셀 파일에 쓰는 함수 호출
write_stat_data_box_plot_to_excel_sheet(excel_file, df_months)

print("생성 파일:", excel_file) # 생성한 파일 이름 출력

## 9.2 통계 데이터 분석 심화

### 상관 분석

**[9장: 531~532페이지]**

In [ ]:
import pandas as pd

#  데이터 파일의 전체 경로
folder = 'C:/myPyExcel/data/ch09/'
csv_file = folder + '기온별_아이스_커피_판매량.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file)

# DataFrame 데이터 출력
df.head()

**[9장: 532페이지]**

In [ ]:
ax_scatter = df.plot.scatter(x='기온', y='아이스커피판매량', grid=True)

ax_scatter.set_xlabel("기온 (섭씨)", fontsize=15)
ax_scatter.set_ylabel("아이스 커피 판매량", fontsize=15)
ax_scatter.set_title("기온별 아이스 커피 판매량", fontsize=20)

plt.show()

**[9장: 533페이지]**

In [ ]:
df.corr() # DataFrma 데이터(df)의 상관 계수 구하기

**[9장: 534~535페이지]**

In [ ]:
import pandas as pd

#  데이터 파일의 전체 경로
folder = 'C:/myPyExcel/data/ch09/'
csv_file = folder + 'iris_setosa_data.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file)

# DataFrame 데이터 출력
df.head()

**[9장: 535페이지]**

In [ ]:
from pandas.plotting import scatter_matrix

# 두 개의 변수 간의 산점도와 한 변수에 대한 히스토그램 표시
# (alpha 옵션은 0~1 사이의 실수로 지정)
scatter_matrix(df, alpha=1, figsize=(8, 8)) 
plt.show()

**[9장: 536페이지]**

In [ ]:
df.corr() # DataFrame 데이터(df)의 상관 계수 구하기

### 회귀 분석

**[9장: 540~541페이지]**

In [ ]:
import pandas as pd

# CSV 파일을 DataFrame으로 읽어옴
folder = 'C:/myPyExcel/data/ch09/'
csv_file = folder + '기온별_아이스_커피_판매량.csv'
df = pd.read_csv(csv_file)
df.head()

**[9장: 541페이지]**

In [ ]:
X = df['기온'].values.reshape(-1,1) # X는 행렬
y = df['아이스커피판매량'].values   # y는 벡터

In [ ]:
from sklearn.linear_model import LinearRegression

# LinearRegression 객체(model)에 대해 fit() 메서드로 회귀 분석 수행
model = LinearRegression().fit(X, y) 

# 선형 회귀 분석의 파라미터 추정값 가져오기
beta0 = model.intercept_
beta1 = model.coef_[0]

# 추정값 결과 출력
print("beta0 = {0:.4f}, beta1 = {1:.4f}".format(beta0, beta1))

**[9장: 542페이지]**

In [ ]:
import numpy as np

# predict() 메서드로 입력 X에 대한 y_hat 추정 
y_hat = model.predict(X)

# SST, SSR, SSE 계산
SST = np.sum((y - np.mean(y))**2)      # SST 계산
SSR = np.sum((y_hat - np.mean(y))**2)  # SSR 계산
SSE = np.sum((y - y_hat)**2)           # SSE 계산

# 결정 계수(R 제곱) 계산
R_squared = 1 - SSE / SST # 결정 계수 계산

# 출력
print("결정 계수: {0:.4f}".format(R_squared))

In [ ]:
from sklearn.metrics import r2_score

r2_score(y, y_hat)

**[9장: 543페이지]**

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# 한글 폰트를 위한 설정
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

# 주어진 데이터 X와 y의 산점도
plt.scatter(X, y) # 산점도 그리기
plt.grid(True) # 격자 추가
plt.xlabel("기온 (섭씨)", fontsize=15)              # x축 라벨 
plt.ylabel("아이스 커피 판매량", fontsize=15)       # y축 라벨
plt.title("기온별 아이스 커피 판매량", fontsize=20) # 그래프 제목

# 추세선 표시
x_data = np.array([10, 40])       # 추세선을 위한 x축 데이터
y_data = beta0 + beta1 * x_data   # 추세선을 위한 y축 데이터

plt.plot(x_data, y_data, 'r--') # y = beta_0 + beta_1 * x에 해당하는 선을 그림

# 추세선 수식을 그래프에 표시
eq_text = "y={0:.4f} + {1:.4f}x".format(beta0, beta1) # 그래프에 표시할 문자열 생성
plt.text(12, 250, eq_text, fontsize=15) # 지정한 위치(x, y) = (12, 250)에 문자열 표시

plt.show()

**[9장: 544페이지]**

In [ ]:
high_temp = [38.5] # 오늘의 최고 기온
X_new = np.array(high_temp).reshape(-1, 1)

# predict() 메서드로 입력 X_new에 대한 y_hat 추정 
y_hat = model.predict(X_new)

# 결과 출력
print("- 최고 기온:", high_temp)
print("- 아이스 커피 예상 판매량:", y_hat)

**[9장: 545페이지]**

In [ ]:
# 새로운 독립 변수 X_new 생성
high_temps = [23.2, 38.5, 39.1] # 여러 날의 최고 기온
X_new = np.array(high_temps).reshape(-1, 1)

# predict() 메서드로 입력 X_new에 대한 y_hat 추정 
y_hat = model.predict(X_new)

# 추정 결과를 이용해 DataFrame 데이터 생성
ice_coffees = y_hat.astype(int) # 실수를 정수로 변환 후 ice_coffees에 할당
df_new = pd.DataFrame({'기온_new': high_temps, 
                       '아이스커피_예상_판매량': ice_coffees})
df_new

**[9장: 548페이지]**

In [ ]:
import pandas as pd

# CSV 파일을 DataFrame으로 읽어옴
folder = 'C:/myPyExcel/data/ch09/'
csv_file = folder + '환경별_수확량.csv'

df = pd.read_csv(csv_file)
df.head()

In [ ]:
import seaborn as sns       # seaborn 임포트

sns.pairplot(df, height=1.8) # height 옵션으로 그래프의 크기를 조정
plt.show()

**[9장: 549페이지]**

In [ ]:
X = df[['온도','습도','CO2']].values # X는 행렬
y = df['수확량'].values              # y는 벡터

**[9장: 549~550페이지]**

In [ ]:
from sklearn.linear_model import LinearRegression

# LinearRegression 객체(model)에 대해 fit() 메서드로 회귀 분석 수행
model = LinearRegression().fit(X, y) 

# 파라미터 추정값 결과 출력
print("- y 절편 상수:", model.intercept_) # beta0
print("- x의 가중치 벡터:", model.coef_)  # [beta1, beta2, beta3]

**[9장: 550페이지]**

In [ ]:
from sklearn.metrics import r2_score

y_hat = model.predict(X)
r2_score(y, y_hat)

In [ ]:
# 새로운 독립 변수 X_new 생성
temp = [17.7, 27.8, 16.8, 15.2, 39.7]      # 온도
humi = [69.2, 69.9, 48.0, 62.3, 37.4]      # 습도
co2 =  [743.7, 839.6, 770.5, 577.6, 839.3] # CO2 농도

X_new = np.array([temp, humi, co2]) # 각 측정값을 행렬(2차원 배열)로 만듦
X_new = X_new.T # n x p 행렬을 만들기 위해 행렬의 전치(transpose)를 수행
X_new

**[9장: 551페이지]**

In [ ]:
# predict() 메서드로 입력 X_new에 대한 y_hat 추정 
y_hat = model.predict(X_new)

# 추정 결과를 이용해 DataFrame 데이터 생성
df_new = pd.DataFrame({'온도': temp, '습도': humi, "CO2": co2, "예상수확량": y_hat})
df_new.style.format('{:.1f}') # DataFrame 데이터의 출력 형식을 지정 

## 9.3 정리